
# Monasca Bootcamp
## Hands on Lab
---
### [Roland Hochmuth](https://www.linkedin.com/in/rolandhochmuth)
email: <roland.hochmuth@hpe.com>
### [Michael Hoppal](https://www.linkedin.com/in/hoppalmichael)
email: <michael.jam.hoppal@hpe.com>

# Agenda
---
* Overview
* Architecture
* Deployment
* API, CLI (python-monascaclient) and client
* Agent
* Developing
* Current status
* What next?


# Architecture
---

# Horizon
---

# Grafana 2
---

# Logging-as-a-Service
---

# Integration with other OpenStack projects
---
* Ceilometer
* Heat
* Congress
* Vitrage
* Broadview
* Neutron
* Rally

# Other Monasca activities at the Austin OpenStack Summit
---

# Deployment
---

# Import libraries
---

In [ ]:
import datetime
import time

# Import libraries use for visualization and analysis
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
import cufflinks as cf

# Import the Monasca and Keystone client
from monascaclient import client
from monascaclient import ksclient

# Initialize the Keystone and Monasca Client
---

In [ ]:
KEYSTONE_URL = 'http://192.168.10.6:5000/v3'
PROJECT_NAME = 'mini-mon'
USERNAME = 'mini-mon'
PASSWORD = 'password'

In [ ]:
# Authenticate to Keystone
keystone_client = ksclient.KSClient(auth_url=KEYSTONE_URL, username=USERNAME, password=PASSWORD)

# Create the Monasca client
monasca_client = client.Client('2_0', keystone_client.monasca_url, token=keystone_client.token)

# Initialize environment variables to use the Monasca CLI
%env OS_PROJECT_NAME=$PROJECT_NAME
%env OS_PASSWORD=$PASSWORD
%env OS_AUTH_URL=$KEYSTONE_URL
%env OS_USERNAME=$USERNAME

# If you are doing development on your local system and running from there then set the URL to the Monasca API
# to override the one returned from Keystone
# %env MONASCA_API_URL=http://127.0.0.1:8070/v2.0

# Initialize Plotly
---

We'll be using Plotly for displaying some graphs later on in this notebook.

In [ ]:
WIDTH = 640
HEIGHT = 768

cf.offline.go_offline()

layout = Layout(
    autosize=False,
    width=WIDTH,
    height=HEIGHT,
    margin=Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ))

# Using the API
---

The Monasca API Specification has the following resources:

* Versions

* Metrics

* Metrics Measurements

* Metrics Statistics

* Metrics Names

* Notification Methods

* Alarm Definitions

* Alarms

* Alarms State History

# Common concepts

# Dimensions
---

* A dictionary of (key, value) pairs that are used to uniquely identify a metric.

* Used to slice and dice metrics when querying.

* Examples: hostname, service, component, region zone, resource_id, ...

# Roles
---

There are three roles in Monasca:

1. user: Allows the client access to all CRUD operations on the API.

2. agent: Can only POST metrics to the API.

3. delegate: Allows the user to POST or query metrics.

# Pagination
---

# Metrics
---

* name (string(255), required) - The name of the metric.

* dimensions ({string(255): string(255)}, optional) - A dictionary consisting of (key, value) pairs used to uniquely identify a metric and slice and dice on.

    * Examples: hostname, region, zone, service, component, process, ...

* timestamp (string, required) - The timestamp in milliseconds from the Epoch.

* value (float, required) - Value of the metric.

* value_meta ({string(255): string(2048)}, optional) - A dictionary consisting of (key, value) pairs used to add information about the value.

    * Examples: status_code, msg

* tenant_id: Tenant ID to create metrics on behalf of.

    * This parameter can be used to submit metrics from one tenant, to another.
    * Requires the delegate role.

# Create metrics
---

In [ ]:
!monasca help metric-create

# POST /v2.0/metrics request body
---

```
{
	name: http_status,
	dimensions: {
		hostname: hostname.domain.com,
		region: uswest,
		zone: 1,
		service: compute
	}
	timestamp: 0, /* milliseconds */
	value: 0.0,
	value_meta: {
		status_code: 500,
		msg: Internal server error
	}
}
```

# List Metrics
---

In [130]:
!monasca help metric-list

usage: monasca metric-list [--name <METRIC_NAME>]
                           [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                           [--starttime <UTC_START_TIME>]
                           [--endtime <UTC_END_TIME>]
                           [--offset <OFFSET LOCATION>]
                           [--limit <RETURN LIMIT>]

List metrics for this tenant.

Optional arguments:
  --name <METRIC_NAME>  Name of the metric to list.
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (pre

# List metrics
---

In [131]:
!monasca metric-list --limit 5

+--------------------------------------+-----------------------+
| name                                 | dimensions            |
+--------------------------------------+-----------------------+
| apache.net.hits                      | apache_host: devstack |
|                                      | apache_port: 80       |
|                                      | component: apache     |
|                                      | hostname: devstack    |
|                                      | service: apache       |
| apache.net.kbytes_sec                | apache_host: devstack |
|                                      | apache_port: 80       |
|                                      | component: apache     |
|                                      | hostname: devstack    |
|                                      | service: apache       |
| apache.net.requests_sec              | apache_host: devstack |
|                                      | apache_port: 80       |
|                        

# List metrics and filter on name, dimensions and starttime
---

In [132]:
!monasca metric-list --name cpu.user_perc --dimensions hostname=devstack --starttime -60 --limit 10


+---------------+---------------------+
| name          | dimensions          |
+---------------+---------------------+
| cpu.user_perc | hostname: devstack  |
|               | service: monitoring |
+---------------+---------------------+


# Create a function to get metrics using the Monasca client
---

In [133]:
def get_metrics(names = [None], dimensions = None, limit=10):
    metrics = []
    for name in names:
        kwargs = {}
        if name is not None:
            kwargs['name'] = name
        if dimensions is not None:
            kwargs['dimensions'] = dimensions
        kwargs['limit'] = limit
        
        # Invoke the Monasca client
        metrics = metrics + monasca_client.metrics.list(**kwargs)
    return metrics

# Metrics Measurements
---

# Query measurements
---

In [134]:
!monasca help measurement-list

usage: monasca measurement-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                [--endtime <UTC_END_TIME>]
                                [--offset <OFFSET LOCATION>]
                                [--limit <RETURN LIMIT>] [--merge_metrics]
                                <METRIC_NAME> <UTC_START_TIME>

List measurements for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to list measurements.
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
             

# Query measurement list
---

In [135]:
!monasca measurement-list --dimensions hostname=devstack --limit 5 cpu.user_perc -120

+---------------+---------------------+--------------------------+--------------+------------+
| name          | dimensions          | timestamp                | value        | value_meta |
+---------------+---------------------+--------------------------+--------------+------------+
| cpu.user_perc | hostname: devstack  | 2016-04-09T17:50:41.000Z |       11.000 |            |
|               | service: monitoring | 2016-04-09T17:50:56.000Z |       10.200 |            |
|               |                     | 2016-04-09T17:51:11.000Z |       11.300 |            |
|               |                     | 2016-04-09T17:51:26.000Z |       10.100 |            |
|               |                     | 2016-04-09T17:51:41.000Z |       10.800 |            |
+---------------+---------------------+--------------------------+--------------+------------+


# Create a function to get measurements using the Monasca Client
---

In [136]:
def get_measurements(metrics, start_time = None, end_time = None, limit=None):
    measurements = []
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=3600)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:
        kwargs = {}
        kwargs['name'] = metric['name']
        kwargs['dimensions'] = metric['dimensions']
        kwargs['start_time'] = start_time
        kwargs['end_time'] = end_time
        
        # Invoke the Monasca client
        measurements.append(monasca_client.metrics.list_measurements(**kwargs))
        
    return measurements

In [137]:
metrics = get_metrics(['cpu.user_perc'])
get_measurements(metrics)

[[{u'columns': [u'timestamp', u'value', u'value_meta'],
   u'dimensions': {u'hostname': u'devstack', u'service': u'monitoring'},
   u'id': u'2016-04-09T19:50:02.000Z',
   u'measurements': [[u'2016-04-09T18:50:29.000Z', 9.7, {}],
    [u'2016-04-09T18:50:44.000Z', 10, {}],
    [u'2016-04-09T18:50:59.000Z', 10.6, {}],
    [u'2016-04-09T18:51:14.000Z', 9.6, {}],
    [u'2016-04-09T18:51:29.000Z', 11.2, {}],
    [u'2016-04-09T18:51:44.000Z', 9.8, {}],
    [u'2016-04-09T18:51:59.000Z', 10.2, {}],
    [u'2016-04-09T18:52:14.000Z', 9.6, {}],
    [u'2016-04-09T18:52:29.000Z', 10, {}],
    [u'2016-04-09T18:52:44.000Z', 10.4, {}],
    [u'2016-04-09T18:52:59.000Z', 10.2, {}],
    [u'2016-04-09T18:53:14.000Z', 10, {}],
    [u'2016-04-09T18:53:29.000Z', 10.2, {}],
    [u'2016-04-09T18:53:44.000Z', 10.1, {}],
    [u'2016-04-09T18:53:59.000Z', 10.5, {}],
    [u'2016-04-09T18:54:14.000Z', 10.3, {}],
    [u'2016-04-09T18:54:29.000Z', 9.9, {}],
    [u'2016-04-09T18:54:44.000Z', 10.2, {}],
    [u'2016-04-0

# Create a function to translate measurements to a Pandas DataFrame
---

In [138]:
def df_from_measurements(measurements):
    '''Returns a DataFrame given measurements'''
    measurement = measurements[0][0]
    m = np.array(measurement['measurements'])  
    timestamps = m[:, measurement['columns'].index('timestamp')]
    df = pd.DataFrame(index = timestamps)  
    
    for measurement in measurements:
        measure = measurement[0]
        m = np.array(measure['measurements'])
        name = measure['name']
        df[name] = m[:, measure['columns'].index('value')]     
    return df

# Query measurements using the Monasca client
---


In [139]:
#metrics = get_metrics(names=['cpu.user_perc', 'cpu.system_perc'])
metrics = get_metrics()
measurements = get_measurements(metrics)

# Display measurements using Plotly
---

In [140]:
df = df_from_measurements(measurements)
df.iplot(subplots=True, shape=(len(df.columns),1), shared_xaxes=True, fill=True)
#df.iplot(kind='line', fill=True,
#         xTitle='Date',
#         yTitle='Value',
#         title='Statistics',
#         layout=layout)
#df.scatter_matrix(world_readable=True)

# Metrics Statistics
---

# Get statistics
---

In [141]:
!monasca help metric-statistics

usage: monasca metric-statistics [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                 [--endtime <UTC_END_TIME>]
                                 [--period <PERIOD>]
                                 [--offset <OFFSET LOCATION>]
                                 [--limit <RETURN LIMIT>] [--merge_metrics]
                                 <METRIC_NAME> <STATISTICS> <UTC_START_TIME>

List measurement statistics for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to report measurement statistics.
  <STATISTICS>          Statistics is one or more (separated by commas) of
                        [AVG, MIN, MAX, COUNT, SUM].
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
    

# Create a function to get statistics using the Monasca client
---

In [142]:
def get_statistics(metrics, statistics=['avg'], interval = 3600, start_time = None, end_time = None):
    statistics_list = []
    
    period = interval / 512
    
    if period < 60:
        period = 60
        
    period = period - period%60
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=interval)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:
        kwargs = {}
        kwargs['statistics'] = statistics
        kwargs['period'] = period
        kwargs['name'] = metric['name']
        kwargs['dimensions'] = metric['dimensions']
        kwargs['start_time'] = start_time
        kwargs['end_time'] = end_time            
        statistics_list.append(monasca_client.metrics.list_statistics(**kwargs))
        
    return statistics_list

# Create a function to translate statistics to a Pandas DataFrame
---

In [143]:
def df_from_statistics(statistics, fn = 'avg', group_by=[]):
    '''Returns a DataFrame given statistics'''
    stat = statistics[0][0]
    m = np.array(stat['statistics'])  
    timestamps = m[:, stat['columns'].index('timestamp')]
    df = pd.DataFrame(index = timestamps)
    
    for statistic in statistics:
        stat = statistic[0]
        m = np.array(stat['statistics'])
        name = stat['name']
        dimensions = stat['dimensions']
        
        for group in group_by:
            name = name + dimensions[group]
            
        df[name] = m[:, stat['columns'].index(fn)]
    return df

# Query statistics using the Monasca client
---

In [144]:
#metrics = get_metrics(names=['cpu.user_perc', 'cpu.system_perc'])
metrics = get_metrics()
statistics = get_statistics(metrics, ['avg'], 3600)


# Display statistics using Plotly
---

In [145]:
metrics = get_metrics(names=['cpu.user_perc', 'cpu.system_perc'])
metrics = get_metrics()
statistics = get_statistics(metrics, ['avg'], 3600)
df = df_from_statistics(statistics)
#df.iplot(subplots=True, shape=(len(df.columns),1), shared_xaxes=True, fill=True,
df.iplot(kind='line', fill=True,
         xTitle='Date',
         yTitle='Value',
         title='Statistics',
         layout=layout)
# df.scatter_matrix(world_readable=True)

In [146]:
df.iplot(kind='box', layout=layout)

# Notification Methods
---

# Alarm Defintions
---

# Alarms
---

# Alarm History
---

# Query the Alarm History
---

In [147]:
!monasca help alarm-history-list

usage: monasca alarm-history-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                  [--starttime <UTC_START_TIME>]
                                  [--endtime <UTC_END_TIME>]
                                  [--offset <OFFSET LOCATION>]
                                  [--limit <RETURN LIMIT>]

List alarms state history.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR format: -120 (previous 120
                        minutes).
  --endtime <UTC_END_TIME>

# Query the alarm history using the Monasca client
---

In [148]:
monasca_client.alarms.history_list(**{})

[]

# Agent
---

# Agent Setup
---

# Developing and Testing
---

* Python and Java codebase

* Monasca DevStack Plugin

* Unit Tests

* Monasca Tempest Tests

* Monasca and OpenStack CI

# Monasca Repos
---
* Monasca is a micro-services message bus based architecture.
* Several repos:
    * monasca-api: both Python and Java  
    * monasca-persister: both Python and Java
    * monasca-thresh: Java
    * monasca-notification: Python
    * monasca-common: both Python and Java
    * monasca-agent: Python
    * monasca-statsd: Python
    * monasca-ui: Python
    * python-monascaclient: Python
    * puppet-monasca: Puppet
    * monasca-log-api

# Monasca DevStack Plugin
---

* DevStack is the primary developmement environment for OpenStack.

    * See http://docs.openstack.org/developer/devstack/

* The Monasca DevStack plugin installs the Monasca Service, Agent, Horizon Panel, and Grafana

* README at, https://github.com/openstack/monasca-api/tree/master/devstack

* Best way to get started is to install Vagrant using the Vagrantfile at, https://github.com/openstack/monasca-api/blob/master/devstack/Vagrantfile.

# Monasca Tempest Tests
---

* [Tempest](http://docs.openstack.org/developer/tempest) is the integration test suite for OpenStack.

* Tempest has an external [Test Plugin Interface](http://docs.openstack.org/developer/tempest/plugin.html) interface that enables anyone to integrate an external test suite.

* There is a [Monasca Tempest Plugin](https://github.com/openstack/monasca-api/tree/master/monasca_tempest_tests)

* Currently, there are around 150 Tempest Tests written for Monasca that run in around 5 minutes.

# Monasca CI
---

* Monasca is fully integrated in the OpenStack CI system.

* Gated jobs are run with each commit on both the Python and Java components through the normal gates and all the Monasca Tempest tests using the Monasca DevStack Plugin.

* Currently, Java is non-voting

# Monasca Weekly Meetings
---

# What's new?
---

* Multiple metrics

* Many enhancements for filtering and sorting resources that return arrays.

# Next Steps
---

* monasca-transform

* monasca-analytics

* monasca-events

* Multiple metrics

# Acknowledgments
---

* Hewlett Packard Enterprise

* Time Warner Cable

* Fujitsu

* Cisco

* NEC

* Cray

* SAP

* The OpenStack Technical Committee: Doug Hellman, Flavio Percoco, Thierry Carrez

# Resources
---

## Cufflinks
* https://plot.ly/ipython-notebooks/cufflinks/
* http://web.quant-platform.com/trial/yves/Plotly_Cufflinks.html

## ipython Slideshow/RISE
* https://github.com/damianavila/RISE
* http://www.slideviper.oquanta.info/tutorial/slideshow_tutorial_slides.html#/3
* http://www.damian.oquanta.info/posts/make-your-slides-with-ipython.html
* https://github.com/damianavila/slideviper_test/blob/gh-pages/tutorial/slideshow_tutorial.ipynb
* http://lab.hakim.se/reveal-js/#/themes

# Thank you

In [151]:
!monasca metric-list --limit 5

+--------------------------------------+-----------------------+
| name                                 | dimensions            |
+--------------------------------------+-----------------------+
| apache.net.hits                      | apache_host: devstack |
|                                      | apache_port: 80       |
|                                      | component: apache     |
|                                      | hostname: devstack    |
|                                      | service: apache       |
| apache.net.kbytes_sec                | apache_host: devstack |
|                                      | apache_port: 80       |
|                                      | component: apache     |
|                                      | hostname: devstack    |
|                                      | service: apache       |
| apache.net.requests_sec              | apache_host: devstack |
|                                      | apache_port: 80       |
|                        

In [150]:
metrics = get_metrics(names=['cpu.user_perc', 'cpu.idle_perc', 'cpu.stolen_perc', 'cpu.system_perc', 'cpu.wait_perc'])
statistics = get_statistics(metrics, ['avg'], 3600)
df = df_from_statistics(statistics)
#df.iplot(subplots=False, shape=(len(df.columns),1), shared_xaxes=True, fill=True)
#df.iplot(kind='line', fill=True,
#         xTitle='Date',
#         yTitle='Value',
#         title='Statistics',
#         layout=layout)
# df.scatter_matrix(world_readable=True)
df.iplot(kind='box', shape=(len(df.columns),1), shared_xaxes=True, fill=True, dimensions=(WIDTH, HEIGHT))

# stress-ng -c 2 -i 1 -m 1 --vm-bytes 128M -t 60s